In [1]:
import math
import random
import string
import sys

import numpy as np
import torch

import matplotlib.pyplot as plt
import src
from envs import build_env
#from src.evaluator import Evaluator
from src.slurm import init_signal_handler, init_distributed_mode
from t2.realtime_trainer import RealtimeTrainer
from src.utils import initialize_exp
from t2.utils import get_parser

from t2.transformer import build_transformer
argv = [
        '--exp_name', 'first_train', 
        '--tasks', 'add_dataset', 
        '--n_enc_layers', '16', 
        '--n_heads', '4', 
        '--sinusoidal_embeddings', 'true', 
        '--num_workers', '4', 
        '--eval_onl', '0', 
        '--save_periodic', '0', 
        '--epoch_size', '10000', 
        '--batch_size', '32', 
        '--dropout', '0.1', 
        '--attention_dropout', '0.1', 
        '--emb_dim', '64', 
        '--bottleneck_dim', '64', 
        '--nn_output', '1', 
        '--input_seq_length', '47', 
        '--share_inout_emb', 'false'
]

parser = get_parser()

params = parser.parse_args(argv)
#params.input_seq_length = len("ABABAGALAMAGAABABAGALAMAGAABABAGALAMAGAABABAG") + 2
print(params)

bs = params.batch_size
#dim = params.emb_dim

init_distributed_mode(params)
logger = initialize_exp(params)

# CPU / CUDA
if params.cpu:
    assert not params.multi_gpu
else:
    assert torch.cuda.is_available()
    print("gpu")
    
src.utils.CUDA = not params.cpu

env = build_env(params)

# modules = build_transformer(env, params, 'cuda')
# trainer = RealtimeTrainer(modules, env, params)

trainers=[]
number_of_models=2
# for i in range(number_of_models):
#     modules = build_transformer(env, params, 'cuda')

#     trainer = RealtimeTrainer(modules, env, params)
    
#     trainers += [trainer]

params.my_device='cuda:0'
trainer = RealtimeTrainer(build_transformer(env, params), env, params)
trainers += [trainer]

params.my_device='cuda:1'

trainer = RealtimeTrainer(build_transformer(env, params), env, params)
trainers += [trainer]



INFO - 01/05/23 22:39:32 - 0:00:00 - ============ Initialized logger ============
INFO - 01/05/23 22:39:32 - 0:00:00 - accumulate_gradients: 1
                                     amp: -1
                                     attention_dropout: 0.1
                                     batch_size: 32
                                     beam_early_stopping: True
                                     beam_eval: False
                                     beam_length_penalty: 1
                                     beam_size: 1
                                     bottleneck_dim: 64
                                     clip_grad_norm: 5
                                     command: python F:\Programs\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py '-f' 'C:\Users\Dmytro\AppData\Roaming\jupyter\runtime\kernel-f01d6e36-580f-40d8-9223-7151741e14da.json' --exp_id "dmytro_job"
                                     cpu: False
                                     debug: False
               

SAIEnvironment created
Namespace(dump_path='./dumped/', exp_name='first_train', save_periodic=0, exp_id='dmytro_job', fp16=False, amp=-1, emb_dim=64, bottleneck_dim=64, nn_output=1, input_seq_length=47, n_enc_layers=16, n_dec_layers=4, n_heads=4, dropout=0.1, attention_dropout=0.1, share_inout_emb=False, sinusoidal_embeddings=True, env_base_seed=0, max_len=512, batch_size=32, optimizer='adam,lr=0.0001', clip_grad_norm=5, epoch_size=10000, max_epoch=100000, stopping_criterion='', validation_metrics='', accumulate_gradients=1, num_workers=4, same_nb_ops_per_batch=False, reload_size=-1, env_name='sai', tasks='add_dataset', beam_eval=False, beam_size=1, beam_length_penalty=1, beam_early_stopping=True, reload_model='', reload_checkpoint='', eval_only=False, eval_verbose=0, eval_verbose_print=False, debug_slurm=False, debug=False, cpu=False, local_rank=-1, master_port=-1)
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World si

INFO - 01/05/23 22:39:33 - 0:00:00 - Number of parameters (transformer): 1598630
INFO - 01/05/23 22:39:33 - 0:00:00 - Found 370 parameters in model.
INFO - 01/05/23 22:39:34 - 0:00:01 - Optimizers: model
INFO - 01/05/23 22:39:34 - 0:00:01 - Number of parameters (transformer): 1598630
INFO - 01/05/23 22:39:34 - 0:00:02 - Found 370 parameters in model.
INFO - 01/05/23 22:39:35 - 0:00:03 - Optimizers: model


In [2]:
#!nvidia-smi

In [3]:
def act(inp, trainer):
    inp = join_sai(inp)
    lst = []
    for _ in range(bs):
        x = inp
        lst += [x]
        
    return trainer.act(lst)[0]

def to_sai_str(p):
    return join_sai(p.data)


def join_sai(data):
    return ' '.join(data)

def str_diff(s1, s2):
    diff = abs(len(s1) - len(s2))
    for i in range(min(len(s1), len(s2))):
        diff += 1 if s1[i] != s2[i] else 0
    return diff


In [4]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
a = join_sai(A)

In [5]:
import time

In [6]:
start_time = time.time()

losses = []
iterations_number = 100

for it in range(iterations_number):
    for trainer in trainers:
            for _ in range(params.batch_size):
                loss = trainer.learn(a, a)

end_time = time.time()
elapsed_time = end_time - start_time
av_time = elapsed_time/iterations_number/len(trainers)

print(f'Av time per iteration per trainer: {elapsed_time/iterations_number:.2f} seconds')


F:\workspace\ai\Transformers1\src\optim.py:72: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\python_arg_parser.cpp:1418.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)
INFO - 01/05/23 22:39:36 - 0:00:04 - learning: av-score=0.035239361702127645, device=cuda:0
INFO - 01/05/23 22:39:37 - 0:00:05 - learning: av-score=1.0472074468085109, device=cuda:1
INFO - 01/05/23 22:39:37 - 0:00:05 - learning: av-score=0.17686170212765953, device=cuda:0
INFO - 01/05/23 22:39:37 - 0:00:05 - learning: av-score=0.3962765957446808, device=cuda:1
INFO - 01/05/23 22:39:38 - 0:00:05 - learning: av-score=0.6522606382978724, device=cuda:0
INFO - 01/05/23 22:39:38 - 0:00:05 - learning: av-score=0.7805851063829788, device=cuda:1
INFO - 01/05/23 22:39:38 - 0:00:05 - learning: av-score=0.9208776595744685, device=cu

INFO - 01/05/23 22:39:54 - 0:00:21 - learning: av-score=0.9574468085106387, device=cuda:1
INFO - 01/05/23 22:39:54 - 0:00:21 - learning: av-score=0.9574468085106387, device=cuda:0
INFO - 01/05/23 22:39:54 - 0:00:21 - learning: av-score=0.9574468085106387, device=cuda:1
INFO - 01/05/23 22:39:54 - 0:00:22 - learning: av-score=0.9574468085106387, device=cuda:0
INFO - 01/05/23 22:39:54 - 0:00:22 - learning: av-score=0.9574468085106387, device=cuda:1
INFO - 01/05/23 22:39:55 - 0:00:22 - learning: av-score=0.9574468085106387, device=cuda:0
INFO - 01/05/23 22:39:55 - 0:00:22 - learning: av-score=0.9574468085106387, device=cuda:1
INFO - 01/05/23 22:39:55 - 0:00:22 - learning: av-score=0.9574468085106387, device=cuda:0
INFO - 01/05/23 22:39:55 - 0:00:23 - learning: av-score=0.9574468085106387, device=cuda:1
INFO - 01/05/23 22:39:55 - 0:00:23 - learning: av-score=0.9574468085106387, device=cuda:0
INFO - 01/05/23 22:39:56 - 0:00:23 - learning: av-score=0.9574468085106387, device=cuda:1
INFO - 01/

INFO - 01/05/23 22:40:12 - 0:00:39 - learning: av-score=0.9574468085106387, device=cuda:1
INFO - 01/05/23 22:40:12 - 0:00:39 - learning: av-score=0.9574468085106387, device=cuda:0
INFO - 01/05/23 22:40:12 - 0:00:40 - learning: av-score=0.9574468085106387, device=cuda:1
INFO - 01/05/23 22:40:12 - 0:00:40 - learning: av-score=0.9574468085106387, device=cuda:0
INFO - 01/05/23 22:40:13 - 0:00:40 - learning: av-score=0.9574468085106387, device=cuda:1
INFO - 01/05/23 22:40:13 - 0:00:40 - learning: av-score=0.9574468085106387, device=cuda:0
INFO - 01/05/23 22:40:13 - 0:00:40 - learning: av-score=0.9587765957446812, device=cuda:1
INFO - 01/05/23 22:40:13 - 0:00:41 - learning: av-score=0.9574468085106387, device=cuda:0
INFO - 01/05/23 22:40:13 - 0:00:41 - learning: av-score=0.9574468085106387, device=cuda:1
INFO - 01/05/23 22:40:14 - 0:00:41 - learning: av-score=0.9574468085106387, device=cuda:0
INFO - 01/05/23 22:40:14 - 0:00:41 - learning: av-score=0.9587765957446812, device=cuda:1
INFO - 01/

Av time per iteration per trainer: 0.40 seconds


In [7]:
next(trainers[0].modules['transformer'].te1.parameters()).device

device(type='cuda', index=0)

In [8]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3080 Ti'

In [9]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

#rnds = [gen_rnd_chars(xy_data_size_const) for _ in range(100)]

print(act(A))
print(act(B))

TypeError: act() missing 1 required positional argument: 'trainer'

In [ ]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

losses = []
for it in range(2000):
    loss = trainer.learn(join_sai(A), join_sai(A))
    if loss[0]:
        l = loss[1].item()
        losses += [l]

plt.plot(losses)
plt.title('los s vs iteration')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.show()


In [ ]:
print(act(A))

In [ ]:
print(act(B))